 读取训练文件

In [1]:
f = open('链路预测数据集/FB15k/freebase_mtr100_mte100-train.txt')
data = f.read()
triples = data.split('\n')
triples = triples[:483142]

totals = list([entity_and_relation for triple in triples for entity_and_relation in triple.split('\t')])
print('三元组数量',len(triples))
print('实体总数',len(set(totals)))
#嵌入的总数为16296

三元组数量 483142
实体总数 16296


生成头实体集合 尾实体集合 关系集合

In [2]:
head_entities = list([triple.split('\t')[0] for triple in triples])
relations =  list([triple.split('\t')[1] for triple in triples])
tail_entities =  list([triple.split('\t')[2] for triple in triples])

total_entities = head_entities + tail_entities

print('实体数',len(set(head_entities + tail_entities)))
print('关系数',len(set(relations)))

实体数 14951
关系数 1345


生成负采样三元组

In [3]:
import random

# 生成负采样的三元组
invalid_head_entities = []
invalid_relations = []
invalid_tail_entities = []

for i in range(len(relations)):
    random_int = random.randint(0,len(head_entities)-1)
    random_int1 = random.randint(0,len(head_entities)-1)
    
    # 不同时替换头尾实体
    if i%2 ==0:
        invalid_head_entities.append(head_entities[i])
        invalid_relations.append(relations[i])
        invalid_tail_entities.append(tail_entities[random_int1])
    else:
        invalid_head_entities.append(head_entities[random_int])
        invalid_relations.append(relations[i])
        invalid_tail_entities.append(tail_entities[i])
#     invalid_head_entities.append(head_entities[random_int])
#     invalid_relations.append(relation)
#     invalid_tail_entities.append(tail_entities[random_int1])

print(head_entities[123456],relations[123456],tail_entities[123456])
print(head_entities[12345],relations[12345],tail_entities[12345])
print(head_entities[12234],relations[12234],tail_entities[12234])

print(invalid_head_entities[123456],invalid_relations[123456],invalid_tail_entities[123456])
print(invalid_head_entities[12345],invalid_relations[12345],invalid_tail_entities[12345])
print(invalid_head_entities[12234],invalid_relations[12234],invalid_tail_entities[12234])

/m/02w2bc /education/educational_institution/students_graduates./education/education/major_field_of_study /m/0g26h
/m/0kpzy /location/us_county/county_seat /m/0dc95
/m/081pw /military/military_conflict/combatants./military/military_combatant_group/combatants /m/0193qj
/m/02w2bc /education/educational_institution/students_graduates./education/education/major_field_of_study /m/0c_dx
/m/035qlx /location/us_county/county_seat /m/0dc95
/m/081pw /military/military_conflict/combatants./military/military_combatant_group/combatants /m/0f1_p


对实体编码

In [4]:
# 为每个实体/关系分配倒排索引
# 根据 名称 查找 id
reverse_index = dict([key,index] for index,key in enumerate(set(totals)))
print('索引长度',len(reverse_index))

#数据标签化

head_entities_encode = list([reverse_index[entry] for entry in head_entities])
relations_encode = list([reverse_index[relation] for relation in relations])
tail_entities_encode = list([reverse_index[entry] for entry in tail_entities])

invalid_head_entities_encode = list([reverse_index[entry] for entry in invalid_head_entities])
invalid_relations_encode = list([reverse_index[relation] for relation in invalid_relations])
invalid_tail_entities_encode = list([reverse_index[entry] for entry in invalid_tail_entities])

print(head_entities_encode[123456],relations_encode[123456],tail_entities_encode[123456])
print(head_entities_encode[12345],relations_encode[12345],tail_entities_encode[12345])
print(head_entities_encode[12234],relations_encode[12234],tail_entities_encode[12234])

print(invalid_head_entities_encode[123456],invalid_relations_encode[123456],invalid_tail_entities_encode[123456])
print(invalid_head_entities_encode[12345],invalid_relations_encode[12345],invalid_tail_entities_encode[12345])
print(invalid_head_entities_encode[12234],invalid_relations_encode[12234],invalid_tail_entities_encode[12234])

索引长度 16296
9688 14075 3513
579 8609 1856
13214 11808 10992
9688 14075 8644
7155 8609 1856
13214 11808 1284


组合数据

In [5]:
# 编码后的数据
golden_data = list(zip(head_entities_encode,relations_encode,tail_entities_encode))
invalid_data = list(zip(invalid_head_entities_encode,invalid_relations_encode,invalid_tail_entities_encode))
    
print(golden_data[123456])
print(invalid_data[123456])

print('三元组数量',len(golden_data))
print('三元组数量',len(invalid_data))

(9688, 14075, 3513)
(9688, 14075, 8644)
三元组数量 483142
三元组数量 483142


获取数据

In [6]:
keep = 0 
def nextbatch(batchsize=400):
    global keep
    start = keep
    end = start + batchsize
    if end > len(golden_data):
        end = end % len(golden_data)
        keep = end
        return golden_data[start:]+golden_data[:end],invalid_data[start:]+invalid_data[:end]
    else:
        keep = end
        return golden_data[start:end],invalid_data[start:end]


[(15918, 12260, 7027), (4340, 15343, 13949), (9041, 8846, 11101), (8369, 8337, 4745)]


In [7]:
import tensorflow as tf
import math
import time

vocabulary_size = 16296
embedding_size = 50
epochs_num = 1
batch_size = 400
margin = 1
learning_rate = 1e-1

#定义模型
def l1_energy(batch):
    return tf.reduce_sum(tf.abs(batch[:, 0, :] + batch[:, 1, :] - batch[:, 2, :]), 1)

embedding_table = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size], stddev=1.0 / math.sqrt(embedding_size)))

pos_triples_batch = tf.placeholder(tf.int32, shape=[batch_size, 3])
neg_triples_batch = tf.placeholder(tf.int32, shape=[batch_size, 3])
predict_triples_batch = tf.placeholder(tf.int32,shape=[1,1])

pos_embed_batch = tf.nn.embedding_lookup(embedding_table, pos_triples_batch)
neg_embed_batch = tf.nn.embedding_lookup(embedding_table, neg_triples_batch)
predict_embed_batch = tf.nn.embedding_lookup(embedding_table, predict_triples_batch)

p_loss = l1_energy(pos_embed_batch)
n_loss = l1_energy(neg_embed_batch)

loss = tf.reduce_sum(tf.nn.relu(margin + p_loss - n_loss))
optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss)

sess = tf.Session()
sess.run(tf.initialize_all_variables())

for step in range(epochs_num):
    time_start = time.time() #开始计时
    loss_total = 0
    print('start:',keep)
    
    for i in range(len(golden_data)//batch_size + 1):
        
        golden_batch,invalid_batch = nextbatch(batchsize=batch_size)    
        feed_dict = {pos_triples_batch: golden_batch, neg_triples_batch: invalid_batch}
        loss_val, _ = sess.run([loss, optimizer], feed_dict=feed_dict)
        loss_total += loss_val
        
    print("\rstep %d, loss_val %f" % (step, loss_total/(len(golden_data)//batch_size + 1)))
    time_end = time.time()    #结束计时
    sum_t= time_end - time_start   #运行所花时间
    print('time cost', sum_t, 's')

Instructions for updating:
Use `tf.global_variables_initializer` instead.
start: 4
step 0, loss_val 185.421199
time cost 9.436769008636475 s


In [ ]:
total_embedding_index = []

for entity in list(reverse_index):
    encode = reverse_index[entity]
    embedding = sess.run([predict_embed_batch], feed_dict= {predict_triples_batch: [[encode]]})[0][0][0]
    total_embedding_index.append([entity,embedding])
    
TransE_index = dict(total_embedding_index)
print(TransE_index['/m/035yzw'])
print('...')
entity_embedding_index = []
for entity in list(total_entities):
    encode = reverse_index[entity]
    embedding = sess.run([predict_embed_batch], feed_dict= {predict_triples_batch: [[encode]]})[0][0][0]
    entity_embedding_index.append([entity,embedding])
print('end')
TransE_entity_index = dict(entity_embedding_index)

链接预测评测代码

In [ ]:
# 读入训练数据
# 先用hit@10标准测试一下
valid_file = open('链路预测数据集/FB15k/freebase_mtr100_mte100-valid.txt')
valid_data = valid_file.read()
valid_triples = valid_data.split('\n')[:50000]

#生成验证集实体和关系集合
valid_head_entities = list([triple.split('\t')[0] for triple in valid_triples])
valid_relations =  list([triple.split('\t')[1] for triple in valid_triples])
valid_tail_entities =  list([triple.split('\t')[2] for triple in valid_triples])

In [ ]:
import numpy as np

hit10 = 0
hit50 = 0
hit1000 = 0
num = 0

def takeFirst(elem):
    return elem[0]

# for head,relation,tail in zip(head_entities[:10],relations[:10],tail_entities[:10]):
for head,relation,tail in zip(valid_head_entities,valid_relations,valid_tail_entities):
    num = num+1
    
    print(head,relation,tail)
    #　头实体嵌入
    valid_head_embedding = TransE_index[head]
    #　关系的嵌入
    valid_relation_embedding = TransE_index[relation]
    #　头尾实体加和
    head_and_relation = valid_head_embedding + valid_relation_embedding
    # 预测一个尾实体
    predict_tail_embedding = head_and_relation
    
    #　与所有的实体计算距离
    distance_list = []
    
    # 遍历TransE嵌入中的所有实体
    for index,entity in enumerate(list(TransE_entity_index)):
        
        #　与TransE中的所有实体进行比较
        embedding = TransE_index[entity]
        
        # 二阶距离
        distance = np.linalg.norm(predict_tail_embedding - embedding)
        distance_list.append([distance,entity])

    # 对所有距离进行排序
    distance_list.sort(key=takeFirst)

    print(distance_list[0])

    predict_1000_list = list([entity[1] for entity in distance_list])[:1000]
    predict_50_list = list([entity[1] for entity in distance_list])[:50]
    predict_10_list = list([entity[1] for entity in distance_list])[:10]

    if tail in predict_1000_list:
        hit1000 = hit1000+1
        
    if tail in predict_50_list:
        hit50 = hit50+1
        
    if tail in predict_10_list:
        hit10 = hit10 +1
        
    print('hit',hit10/num,hit50/num,hit1000/num,num)

开始预测
/m/07pd_j /film/film/genre /m/02l7c8
[2.3143082, '/m/0clz1b']
hit 0.0 1.0 1.0 1
/m/06wxw /location/location/time_zones /m/02fqwt
[2.0801835, '/m/06wxw']
hit 0.0 1.0 1.0 2
/m/0d4fqn /award/award_winner/awards_won./award/award_honor/award_winner /m/03wh8kl
[0.6346668, '/m/0d4fqn']
hit 0.0 0.6666666666666666 0.6666666666666666 3
/m/07kcvl /american_football/football_team/historical_roster./american_football/football_historical_roster_position/position_s /m/0bgv8y
[1.910367, '/m/01y_px']
hit 0.0 0.5 0.75 4
/m/012201 /film/music_contributor/film /m/0ckrnn
[1.7819847, '/m/067ghz']
hit 0.0 0.4 0.6 5
/m/018gz8 /people/profession/people_with_this_profession /m/01520h
[2.0492878, '/m/019pkm']
hit 0.0 0.3333333333333333 0.5 6
/m/02x17c2 /award/award_category/nominees./award/award_nomination/award_nominee /m/0x3b7
[1.8606318, '/m/028mc6']
hit 0.0 0.2857142857142857 0.42857142857142855 7
/m/03swmf /people/person/education./education/education/major_field_of_study /m/06nm1
[1.6538703, '/m/0l5mz

[1.6081743, '/m/0cg9y']
hit 0.14545454545454545 0.21818181818181817 0.6181818181818182 55
/m/071h5c /soccer/football_player/position_s /m/0dgrmp
[1.6564357, '/m/0ccvd']
hit 0.14285714285714285 0.21428571428571427 0.6071428571428571 56
/m/0g56t9t /film/film/genre /m/05p553
[2.4301372, '/m/03npn']
hit 0.14035087719298245 0.21052631578947367 0.6140350877192983 57
/m/03vtrv /music/record_label/artist /m/04rcr
[2.2572486, '/m/03vtrv']
hit 0.13793103448275862 0.20689655172413793 0.603448275862069 58
/m/024tcq /government/legislative_session/members./government/government_position_held/district_represented /m/07b_l
[1.6405853, '/m/050l8']
hit 0.13559322033898305 0.22033898305084745 0.6101694915254238 59
/m/0kjgl /people/person/parents /m/0j582
[1.5541296, '/m/0kjgl']
hit 0.13333333333333333 0.23333333333333334 0.6166666666666667 60
/m/016lmg /common/topic/webpage./common/webpage/category /m/08mbj5d
[1.991959, '/m/08mbj5d']
hit 0.14754098360655737 0.2459016393442623 0.6229508196721312 61
/m/03

[1.4739343, '/m/0p_sc']
hit 0.16666666666666666 0.25 0.6089743589743589 156
/m/01h8f /common/topic/webpage./common/webpage/category /m/08mbj5d
[2.2283618, '/m/08mbj32']
hit 0.17197452229299362 0.25477707006369427 0.6114649681528662 157
/m/026mj /location/location/contains /m/0rgxp
[1.6337129, '/m/049n7']
hit 0.17088607594936708 0.25316455696202533 0.6075949367088608 158
/m/014mlp /education/educational_degree/people_with_this_degree./education/education/student /m/03ldxq
[1.1910335, '/m/01_rh4']
hit 0.16981132075471697 0.25157232704402516 0.6037735849056604 159
/m/02t_tp /film/actor/film./film/performance/film /m/0571m
[1.9888835, '/m/05q96q6']
hit 0.16875 0.25 0.6 160
/m/02wgk1 /award/award_nominated_work/award_nominations./award/award_nomination/award /m/057xs89
[2.5678585, '/m/09sdmz']
hit 0.16770186335403728 0.2546583850931677 0.6024844720496895 161
/m/0ct2tf5 /film/film/distributors./film/film_film_distributor_relationship/distributor /m/016tw3
[2.1809757, '/m/024rdh']
hit 0.17283

hit 0.16990291262135923 0.27184466019417475 0.6359223300970874 206
/m/04bdzg /base/popstra/celebrity/dated./base/popstra/dated/participant /m/0391jz
[1.1399107, '/m/04bdzg']
hit 0.16908212560386474 0.27053140096618356 0.6328502415458938 207
/m/07t90 /education/educational_institution/students_graduates./education/education/major_field_of_study /m/04gb7
[1.8151585, '/m/01jzxy']
hit 0.16826923076923078 0.27403846153846156 0.6346153846153846 208
/m/03h3vtz /award/award_winner/awards_won./award/award_honor/award /m/0cqhk0
[2.2975228, '/m/01ckcd']
hit 0.1674641148325359 0.2727272727272727 0.6363636363636364 209
/m/0qmjd /film/film/directed_by /m/01_vfy
[1.6797376, '/m/05pzdk']
hit 0.16666666666666666 0.2714285714285714 0.638095238095238 210
/m/0glt670 /music/genre/artists /m/01vw37m
[1.773252, '/m/03bnv']
hit 0.16587677725118483 0.27014218009478674 0.6350710900473934 211
/m/0chgzm /travel/travel_destination/climate./travel/travel_destination_monthly_climate/month /m/040fb
[1.7714906, '/m/05

[1.3776181, '/m/0r1jr']
hit 0.14901960784313725 0.25882352941176473 0.6235294117647059 255
/m/026dqjm /sports/sports_team/roster./sports/sports_team_roster/position /m/0619m3
[2.067289, '/m/0nmj']
hit 0.1484375 0.2578125 0.62109375 256
/m/0gt_0v /music/genre/artists /m/01lcxbb
[2.1402583, '/m/06msq']
hit 0.14785992217898833 0.25680933852140075 0.6186770428015564 257
/m/04n1hqz /soccer/football_team/current_roster./sports/sports_team_roster/position /m/02sdk9v
[1.8997133, '/m/0dgrmp']
hit 0.1511627906976744 0.2596899224806202 0.6201550387596899 258
/m/0cms7f /award/award_nominee/award_nominations./award/award_nomination/award_nominee /m/043js
[0.70927143, '/m/0cms7f']
hit 0.15057915057915058 0.25868725868725867 0.6216216216216216 259
/m/0b7l1f /sports/pro_athlete/sports_played_professionally./sports/pro_sports_played/sport /m/02vx4
[1.5207111, '/m/0b7l1f']
hit 0.15384615384615385 0.26153846153846155 0.6230769230769231 260
/m/0fdys /education/field_of_study/students_majoring./education/e

...

[1.9226528, '/m/06sks6']
hit 0.159971176364619 0.27994955863808324 0.6218699333453431 11102
/m/013w2r /music/artist/genre /m/05bt6j
[2.256303, '/m/06by7']
hit 0.1599567684409619 0.28001441051967935 0.6219039899126362 11103
/m/09qftb /award/award_ceremony/awards_presented./award/award_honor/award /m/054knh
[2.4356282, '/m/025m98']
hit 0.15994236311239193 0.2799891930835735 0.6218479827089337 11104
/m/024vjd /award/award_category/winners./award/award_honor/award_winner /m/015rmq
[1.8423369, '/m/0pj8m']
hit 0.159927960378208 0.279963980189104 0.6217919855920756 11105
/m/03_d0 /education/field_of_study/students_majoring./education/education/degree /m/016t_3
[2.3508523, '/m/03_d0']
hit 0.15991356023770933 0.2799387718350441 0.62182603997839 11106
/m/0h2zvzr /film/film/executive_produced_by /m/0d0xs5
[1.6512293, '/m/04r7jc']
hit 0.15989916269019538 0.2799135680201675 0.6217700549203206 11107
/m/0yzvw /award/award_winning_work/awards_won./award/award_honor/award /m/09cm54
[2.331662, '/m/05b16

hit 0.1600609756097561 0.2803084648493544 0.6221305595408895 11152
/m/09r1j5 /soccer/football_player/current_team./sports/sports_team_roster/team /m/04mp75
[1.5451103, '/m/09r1j5']
hit 0.16004662422666546 0.2802833318389671 0.6221644400609702 11153
/m/08fn5b /film/film/other_crew./film/film_crew_gig/film_crew_role /m/09vw2b7
[2.5735729, '/m/09zzb8']
hit 0.1601219293526986 0.2803478572709342 0.6221983145060068 11154
/m/02lfns /film/actor/film./film/performance/film /m/048htn
[1.9806504, '/m/05q96q6']
hit 0.16010757507844017 0.2803227252353205 0.6221425369789332 11155
/m/03m5k /music/performance_role/track_performances./music/track_contribution/role /m/05148p4
[2.0392454, '/m/03m5k']
hit 0.1600932233775547 0.2802975977052707 0.6221764073144497 11156
/m/03t79f /film/film/other_crew./film/film_crew_gig/film_crew_role /m/015h31
[2.6717238, '/m/0ch6mp2']
hit 0.1600788742493502 0.28027247467957334 0.6222102715783813 11157
/m/02ctyy /film/person_or_entity_appearing_in_film/films./film/personal

/m/02r251z /film/producer/film /m/0408m53
[1.4172024, '/m/03f7xg']
hit 0.15995715433366062 0.2799250200839061 0.6224225653842721 11203
/m/08mbj32 /common/annotation_category/annotations./common/webpage/topic /m/05tg3
[1.2363435, '/m/06khd']
hit 0.15994287754373437 0.27990003570153515 0.6223670117815066 11204
/m/045c66 /people/person/profession /m/02hrh1q
[2.2782114, '/m/0dz3r']
hit 0.15992860330209727 0.27996430165104863 0.622400713966979 11205
/m/0147fv /film/special_film_performance_type/film_performance_type./film/performance/actor /m/0k525
[1.3386934, '/m/06w6_']
hit 0.15991433160806712 0.2799393182223809 0.622345172229163 11206
/m/0gqy2 /award/award_category/nominees./award/award_nomination/nominated_for /m/0kbf1
[2.3274245, '/m/0c5qvw']
hit 0.1599000624609619 0.27991433925225306 0.6223788703488891 11207
/m/02vxfw_ /film/film_job/films_with_this_crew_job./film/film_crew_gig/film /m/071nw5
[1.9922501, '/m/0cw3yd']
hit 0.15988579586009993 0.2798893647394718 0.6223233404710921 11208


[1.6352683, '/m/03zv3n']
hit 0.15980801706514977 0.2794418273931206 0.6221669184961337 11251
/m/01w9k25 /award/award_nominee/award_nominations./award/award_nomination/award_nominee /m/01vvydl
[0.70927143, '/m/01w9k25']
hit 0.15979381443298968 0.2794169925346605 0.6221116246000711 11252
/m/027cxsm /award/award_nominee/award_nominations./award/award_nomination/award_nominee /m/0bt4r4
[0.70927143, '/m/027cxsm']
hit 0.15977961432506887 0.2793921620901093 0.6220563405314139 11253
/m/0x67 /people/ethnicity/people /m/014gf8
[1.3910388, '/m/04g3p5']
hit 0.1597654167407144 0.2793673360582904 0.6220010662875423 11254
/m/03qjg /music/performance_role/regular_performances./music/group_membership/group /m/0187x8
[1.8708712, '/m/0163m1']
hit 0.15975122167925365 0.27934251443802754 0.6219458018658374 11255
/m/05ldnp /film/writer/film /m/03r0g9
[1.7395622, '/m/0c1sgd3']
hit 0.15973702914001423 0.279317697228145 0.6218905472636815 11256
/m/0gr4k /award/award_category/nominees./award/award_nomination/aw

[1.9270393, '/m/02mxx4']
hit 0.15948314010089387 0.27931675369501724 0.6215594300380565 11299
/m/043tz8m /sports/sports_team/roster./sports/sports_team_roster/position /m/06b1q
[1.7713436, '/m/08ns5s']
hit 0.15946902654867257 0.2792920353982301 0.6215929203539823 11300
/m/01795t /award/award_winner/awards_won./award/award_honor/award /m/02x1z2s
[2.2849786, '/m/02w7fs']
hit 0.15945491549420404 0.2792673214759756 0.6216264047429431 11301
/m/01ksr1 /award/award_winner/awards_won./award/award_honor/honored_for /m/011ywj
[1.4775523, '/m/0pd4f']
hit 0.15944080693682533 0.27924261192709254 0.6216598832065121 11302
/m/05vsb7 /sports/sports_league_draft/picks./sports/sports_league_draft_pick/team /m/084l5
[1.7132947, '/m/01l3j']
hit 0.15942670087587366 0.27921790675042024 0.621693355746262 11303
/m/01vsnff /music/artist/label /m/041n43
[1.8779379, '/m/01cszh']
hit 0.1594125973106865 0.2791932059447983 0.6217268223637651 11304
/m/04x1_w /people/person/places_lived./people/place_lived/location /m